<a href="https://colab.research.google.com/github/dhruthick/cse256project/blob/main/data/interactions_data_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
interactions=pd.read_csv('/content/drive/MyDrive/cse256/project/data/interactions.csv', nrows=500000)[['pid','track_uri','position']]
tracks=pd.read_csv('/content/drive/MyDrive/cse256/project/data/interactions_with_track_name.csv')[['track_name','track_uri','artist_name','duration','artist_uri','album_uri','album_name']]

In [3]:
interactions=interactions.join(tracks.set_index('track_uri'),on='track_uri',how='inner').reset_index(drop=True)

In [4]:
playlist_lengths=interactions.pid.value_counts().to_dict()
drop_playlists=[p for p,l in playlist_lengths.items() if l<1]
interactions = interactions[~interactions['pid'].isin(drop_playlists)]

In [12]:
pid2gen_pid={}
track_uri2id={}
pid=0
for playlist in np.unique(interactions.pid):
  pid2gen_pid[playlist]=pid
  pid+=1
tid=0
for track in np.unique(interactions.track_uri):
  track_uri2id[track]=tid
  tid+=1

In [18]:
interactions['gen_pid']=interactions.pid.apply(lambda x:pid2gen_pid[x])
interactions['tid']=interactions.track_uri.apply(lambda x:track_uri2id[x])

In [20]:
all_playlists=np.unique(interactions.gen_pid)

In [21]:
train_playlists=all_playlists[:int(len(all_playlists)*0.75)]
val_playlists=all_playlists[int(len(all_playlists)*0.75):int(len(all_playlists)*0.90)]
test_playlists=all_playlists[int(len(all_playlists)*0.90):]

print(f'Size of train, val and test sets: {len(train_playlists),len(val_playlists),len(test_playlists)}')

Size of train, val and test sets: (5607, 1121, 748)


In [22]:
interactions[interactions['pid'].isin(train_playlists)].reset_index(drop=True).to_csv('/content/drive/MyDrive/cse256/project/data/interactions_train.csv',index=False)
interactions[interactions['pid'].isin(val_playlists)].reset_index(drop=True).to_csv('/content/drive/MyDrive/cse256/project/data/interactions_val.csv',index=False)
interactions[interactions['pid'].isin(test_playlists)].reset_index(drop=True).to_csv('/content/drive/MyDrive/cse256/project/data/interactions_test.csv',index=False)